In [ ]:
%load_ext autoreload
%autoreload 2

from getAttentionLib import ActivationPatchingResult, load_vqa_ds, plot_squared_imgs
from getAttentionLib import plot_img_and_text_probs_side_by_side
from getAttentionLib import load_vqa_samples, unique_vqa_imgs
from getAttentionLib import load_pg2_model_and_processor
import pandas as pd
import matplotlib.pyplot as plt

is_gaussian_noising = True
pr = ActivationPatchingResult.load("vqa_patching/str/008")
dataset = load_vqa_ds(split="train")
img_ids = pd.Series(dataset["image_id"])

In [ ]:
print(
    (
        pr.metadata["prompt"],
        pr.metadata["healthy_response_tok"],
        pr.metadata["unhealthy_response_tok"],
    )
)
if pr.metadata["corruption_img_alias"] is not None:
    rows = dataset.select(
        [
            img_ids[img_ids == pr.metadata["healthy_img_alias"]].index.values[0],
            img_ids[img_ids == pr.metadata["corruption_img_alias"]].index.values[0],
        ]
    )
    hvqa_row = rows[0]
    uvqa_row = rows[1]
    plot_squared_imgs(hvqa_row["image"], uvqa_row["image"]).show()
else:
    row = dataset.select(
        [img_ids[img_ids == pr.metadata["healthy_img_alias"]].index.values[0]]
    )[0]
    plt.imshow(row["image"])


plot_img_and_text_probs_side_by_side(
    probs=pr.healthy_tok_response_probs,
    n_img_tokens=n_img_tokens,
    token_strings=token_strings,
    token_str=pr.metadata["healthy_response_tok"],
).show()

plot_img_and_text_probs_side_by_side(
    probs=pr.unhealthy_tok_response_probs,
    n_img_tokens=n_img_tokens,
    token_strings=token_strings,
    token_str=pr.metadata["unhealthy_response_tok"],
).show()

In [ ]:
n_img_tokens = 256
token_strings = pr.metadata["token_strings"]
logit_diff = pr.get_logit_diff_gn() if is_gaussian_noising else pr.get_logit_diff()
plot_img_and_text_probs_side_by_side(
    logit_diff,
    n_img_tokens=n_img_tokens,
    token_strings=token_strings,
    token_str=pr.metadata["healthy_response_tok"],
    is_probabilities=False,
).show()

import os
from pathlib import Path
from getAttentionLib import dump_activation_patching_for_sample


root = Path("vqa_patching")
for dirname in os.listdir(root):
    pr = ActivationPatchingResult.load(root / dirname)
    dump_activation_patching_for_sample(
        pr=pr,
        dataset=dataset,
        n_img_tokens=n_img_tokens,
        tgt_dir=f"imgs/vqa_patching/cases/{dirname}"
    )


# Interesting cases:
* IN 005. Question=the man smiling? healthy_img=man not smiling, unhealthy_img=2 men with bear. The stronges restoration besides the last layer&token is the "smiling" token in intermediate layer.
Mind that the question has not changed, only its internal representation changed, meaning that the models is accumulating image information in the text tokens.
* A similar thing happens in 000: the text token "thorugh" restores partially the clean response. The image tokens play role here as well.
* in image 008 we can also see the token "jacket" being restorative
* In image 020, there are counter-intuitive negative effects on restoring the clean response for th "umbrella" token. The model becomes more confident in the wrong answer (white) rather than in the correct one (pink). Same in image 021.

In the paired VQAv2 images:
* In 024 we see a strong negative activation for the "tree" token

self = pr
unhealthy_logit_diff = (
    self.metadata["unhealthy_run_healthy_tok_logit"]
    - self.metadata["unhealthy_run_unhealthy_tok_logit"]
)
patched_logit_diff = (
    self.healthy_tok_response_logits - self.unhealthy_tok_response_logits
)
healthy_logit_diff = (
    self.metadata["healthy_run_healthy_tok_logit"]
    - self.metadata["healthy_run_unhealthy_tok_logit"]
)
normalized = (patched_logit_diff - unhealthy_logit_diff) / (
    healthy_logit_diff - unhealthy_logit_diff
)

layer_idx = 10
token_idx = -3
(
    healthy_logit_diff,
    unhealthy_logit_diff,
    patched_logit_diff[layer_idx, token_idx],
    self.healthy_tok_response_logits[layer_idx, token_idx],
    self.unhealthy_tok_response_logits[layer_idx, token_idx],
    normalized[layer_idx, token_idx],
)